In [1]:
# imports
import sys
import os
import socket
import pandas as pd
import glob
from datetime import datetime

In [2]:
# read in hes dx file
dementia_patients_hes = pd.read_parquet(os.getcwd() + '/dementia_hes_diag_data.parquet')

In [3]:
dementia_patients_hes.head()

,e_patid,icd,epistart,epiend,is_op
178,100007617,F00.9,2010-10-20,2010-10-21,0.0
186,100007617,F00.9,2011-06-19,2011-06-19,0.0
189,100007617,G30.9,2010-10-20,2010-10-21,0.0
207,100007617,G30.9,2011-06-19,2011-06-19,0.0
222,100007617,F01.9,2014-03-27,2014-04-07,0.0


In [4]:
# get unique e_patids
e_patids = dementia_patients_hes['e_patid'].unique()

In [5]:
dementia_patients_hes['epistart'].isna().sum()

388633

In [6]:
dementia_patients_hes['epiend'].isna().sum()

388646

In [7]:
# Drop NaNs
dementia_patients_hes = dementia_patients_hes.dropna()

In [8]:
dementia_patients_hes['epistart'].isna().sum()

0

In [9]:
# get first dx date for each e_patid
first_dementia_code_dates = []
for i in e_patids:
    first_dementia_code_dates += [min(dementia_patients_hes[dementia_patients_hes['e_patid'] == i]['epistart'])]

In [10]:
# make df
df_first_dates = pd.DataFrame({"e_patid": e_patids, "epistart": first_dementia_code_dates})

In [11]:
df_first_dates.head()

,e_patid,epistart
0,100007617,2010-07-29
1,100012174,2015-01-09
2,100013188,2011-11-18
3,100016729,2000-08-08
4,100024652,2010-12-31


In [12]:
# get first dx row for each e_patid by merging
first_dementia_records_hes = df_first_dates.merge(
    right = dementia_patients_hes, on = ['e_patid', 'epistart'], how = 'inner')

In [13]:
first_dementia_records_hes.shape[0]

232926

In [14]:
# save as parquet
first_dementia_records_hes.to_parquet(os.getcwd() + '/first_dementia_records_in_hes.parquet')

In [15]:
first_dementia_records_hes.head()

,e_patid,epistart,icd,epiend,is_op
0,100007617,2010-07-29,F00.9,2010-07-29,0.0
1,100007617,2010-07-29,G30.9,2010-07-29,0.0
2,100012174,2015-01-09,G30.8,2015-01-20,0.0
3,100012174,2015-01-09,F00.2,2015-01-20,0.0
4,100013188,2011-11-18,F03,2011-11-19,0.0
